## DIA 034: Implementación de un Sistema de Feedback de Predicciones

En este día, vamos a añadir una funcionalidad que permita a los usuarios proporcionar retroalimentación sobre las predicciones realizadas por la API. Este sistema de feedback es fundamental para monitorear y mejorar el desempeño del modelo de predicción, ya que permite recolectar datos reales sobre la precisión y utilidad de las predicciones. Utilizaremos SQLAlchemy para definir un nuevo modelo llamado Feedback, que almacenará la información enviada por los usuarios. Además, crearemos endpoints para que los usuarios (autenticados mediante JWT) puedan enviar su feedback y, opcionalmente, para que los administradores puedan consultar estadísticas resumidas de la retroalimentación.

Código Completo (api.py)
python
Copiar
from flask import Flask, Blueprint, request, jsonify, render_template, url_for
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
from flask_bcrypt import Bcrypt
from flask_mail import Mail, Message
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address
import os
import logging
from datetime import datetime
import json

# Configuración básica y variables de entorno
app = Flask(__name__)
app.config['SECRET_KEY'] = os.getenv('SECRET_KEY', 'your_secret_key')
app.config['JWT_SECRET_KEY'] = os.getenv('JWT_SECRET_KEY', 'your_jwt_secret_key')
app.config['SQLALCHEMY_DATABASE_URI'] = os.getenv('DATABASE_URL', 'sqlite:///app.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# Inicialización de extensiones
db = SQLAlchemy(app)
migrate = Migrate(app, db)
bcrypt = Bcrypt(app)
mail = Mail(app)
jwt = JWTManager(app)
limiter = Limiter(app, key_func=get_remote_address, default_limits=["200 per day", "50 per hour"])

# Configuración de Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ---------------------------
# Modelos
# ---------------------------
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    # Otros campos omitidos para simplificar

class Feedback(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), nullable=False)  # Usuario que envía el feedback
    prediction = db.Column(db.Integer, nullable=False)   # Predicción realizada (por ejemplo, dígito)
    correct = db.Column(db.Boolean, nullable=False)        # Indicador de si la predicción fue correcta
    comment = db.Column(db.Text, nullable=True)            # Comentario adicional del usuario
    timestamp = db.Column(db.DateTime, default=datetime.utcnow)  # Momento del feedback

    def to_dict(self):
        return {
            "id": self.id,
            "username": self.username,
            "prediction": self.prediction,
            "correct": self.correct,
            "comment": self.comment,
            "timestamp": self.timestamp.isoformat()
        }

# ---------------------------
# Endpoints de Autenticación (Ejemplo básico)
# ---------------------------
@app.route('/login', methods=['POST'])
def login():
    data = request.get_json()
    username = data.get('username')
    password = data.get('password')
    if not username or not password:
        return jsonify({"msg": "Username and password required"}), 400
    # Para el ejemplo, asumimos que el usuario existe y la contraseña es correcta.
    access_token = create_access_token(identity=username)
    logger.info(f"Usuario '{username}' inició sesión.")
    return jsonify(access_token=access_token), 200

# ---------------------------
# Endpoint para Enviar Feedback
# ---------------------------
@app.route('/feedback', methods=['POST'])
@jwt_required()
def submit_feedback():
    """
    Permite a un usuario autenticado enviar feedback sobre una predicción.
    Se espera un JSON con:
      - prediction: entero (la predicción realizada)
      - correct: booleano (si la predicción fue correcta)
      - comment: (opcional) comentario adicional
    """
    current_user = get_jwt_identity()
    data = request.get_json()
    prediction = data.get('prediction')
    correct = data.get('correct')
    comment = data.get('comment', "")

    if prediction is None or correct is None:
        logger.warning(f"Feedback fallido: Datos incompletos del usuario '{current_user}'.")
        return jsonify({"msg": "Prediction and correct flag required"}), 400

    feedback = Feedback(
        username=current_user,
        prediction=prediction,
        correct=correct,
        comment=comment
    )
    db.session.add(feedback)
    db.session.commit()
    logger.info(f"Feedback registrado por '{current_user}': {feedback.to_dict()}")
    return jsonify({"msg": "Feedback submitted successfully"}), 201

# ---------------------------
# Endpoint para Obtener Feedback Resumido (Solo Admin, por ejemplo)
# ---------------------------
@app.route('/feedback/summary', methods=['GET'])
@jwt_required()
def feedback_summary():
    """
    Devuelve un resumen del feedback recibido, por ejemplo, el total de feedbacks,
    porcentaje de aciertos y errores, etc.
    """
    # En un caso real, se debería verificar que el usuario tiene rol de administrador.
    total_feedback = Feedback.query.count()
    if total_feedback == 0:
        return jsonify({"msg": "No feedback available"}), 200

    correct_feedback = Feedback.query.filter_by(correct=True).count()
    incorrect_feedback = Feedback.query.filter_by(correct=False).count()
    summary = {
        "total_feedback": total_feedback,
        "correct_feedback": correct_feedback,
        "incorrect_feedback": incorrect_feedback,
        "accuracy_percentage": (correct_feedback / total_feedback) * 100
    }
    logger.info("Feedback summary solicitado.")
    return jsonify(summary), 200

# ---------------------------
# Endpoint de Health Check
# ---------------------------
@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok"}), 200

# ---------------------------
# Ejecutar la aplicación
# ---------------------------
if __name__ == '__main__':
    app.run(debug=True)
Explicación del Código
Modelos:

Se añade un nuevo modelo Feedback con los campos necesarios para almacenar la retroalimentación del usuario: username, prediction, correct, comment y timestamp.
La función to_dict() en el modelo facilita la conversión de un objeto de feedback a un diccionario para su devolución en formato JSON.
Endpoints de Autenticación:

Se mantiene un endpoint básico de /login que genera un token JWT para los usuarios. Este token se usará para autenticar solicitudes a otros endpoints, incluido el de feedback.
Endpoint para Enviar Feedback (/feedback):

Requiere que el usuario esté autenticado mediante el decorador @jwt_required().
Espera un JSON que contenga la predicción realizada, un indicador booleano correct y opcionalmente un comentario.
Guarda el feedback en la base de datos y retorna un mensaje de confirmación.
Endpoint para Obtener un Resumen de Feedback (/feedback/summary):

Permite a los usuarios (o administradores) obtener un resumen de la retroalimentación, incluyendo el total de feedbacks y el porcentaje de aciertos.
Se utiliza para evaluar el desempeño del modelo basándose en los datos reales de feedback.
Endpoint de Health Check (/health):

Un endpoint simple para verificar que la aplicación se está ejecutando correctamente.
Ejecución de la Aplicación:

La aplicación se inicia en modo debug cuando se ejecuta directamente.